! pip install psycopg2

In [1]:
import os
import psycopg2

# creat database

In [ ]:
DATABASE_URL = os.popen('heroku config:get DATABASE_URL -a dennis89linebot').read()[:-1]
# DATABASE_URL = os.environ['DATABASE_URL'] #on heroku
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

create_table_query = '''CREATE TABLE alpaca_training(
   record_no serial PRIMARY KEY,
   alpaca_name VARCHAR (50) NOT NULL,
   training VARCHAR (50) NOT NULL,
   duration INTERVAL NOT NULL,
   date DATE NOT NULL
);'''

cursor.execute(create_table_query)
conn.commit()

cursor.close()
conn.close()

# list all table

In [14]:
DATABASE_URL = os.popen('heroku config:get DATABASE_URL -a dennis89linebot').read()[:-1]
# DATABASE_URL = os.environ['DATABASE_URL'] #on heroku
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()
cursor.execute("""SELECT table_name FROM information_schema.tables
       WHERE table_schema = 'public'""")
for table in cursor.fetchall():
    print(table)
cursor.close()
conn.close()

('alpaca_training',)
('apscheduler_jobs',)


# fetch data type

In [15]:
DATABASE_URL = os.popen('heroku config:get DATABASE_URL -a dennis89linebot').read()[:-1]

conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

cursor.execute("SELECT column_name, data_type FROM information_schema.columns WHERE table_name = 'apscheduler_jobs'")
data = []
while True:
    temp = cursor.fetchone()
    if temp:
        data.append(temp)
    else:
        break
print(data)
cursor.close()
conn.close()

[('id', 'character varying'), ('next_run_time', 'double precision'), ('job_state', 'bytea')]


# insert data

In [4]:
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

records = [('大衛', '肌力訓練', '1:00:00', '2019-09-24'),
           ('威廉', '咬合訓練', '1:00:00', '2019-09-24'),
           ('威廉', '牧草訓練', '1:30:00', '2019-09-24'),
           ('彼得', '牧草進食', '1:30:00', '2019-09-24')]
table_columns = '(alpaca_name, training, duration, date)'
postgres_insert_query = f"""INSERT INTO alpaca_training {table_columns} VALUES (%s, %s, %s, %s);"""

cursor.executemany(postgres_insert_query, records) # use executemany for mulitple, but execute for single
conn.commit()

count = cursor.rowcount

print(count, "Record inserted successfully into alpaca_training")

cursor.close()
conn.close()

4 Record inserted successfully into alpaca_training


# fetch data

In [16]:
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

postgres_select_query = f"""SELECT * FROM apscheduler_jobs"""

cursor.execute(postgres_select_query)
# cursor.fetchone()
# print(cursor.fetchmany(2))
print(cursor.fetchall())
cursor.close()
conn.close()

[('cron_job1', 1623925800.0, <memory at 0x0000024B8F49E7C0>), ('cron_job2', 1623942000.0, <memory at 0x0000024B8F49E880>), ('cron_job3', 1623972600.0, <memory at 0x0000024B8F49E940>), ('cron_job5', 1623978060.0, <memory at 0x0000024B8F49E4C0>), ('2021-06-17 21:10:00', 1623935400.0, <memory at 0x0000024B8F49E580>)]


# search data

In [6]:
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

training = '咬合訓練'
postgres_select_query = f"""SELECT * FROM alpaca_training WHERE training = %s"""

cursor.execute(postgres_select_query, (training,))
print(cursor.fetchall())

[(2, '威廉', '咬合訓練', datetime.timedelta(seconds=3600), datetime.date(2019, 9, 24))]


# update data

In [7]:
 conn = psycopg2.connect(DATABASE_URL, sslmode='require')
 cursor = conn.cursor()

 training = '牧草訓練'
 postgres_update_query = f"""UPDATE alpaca_training SET training = '牧草進食' WHERE training = %s"""

 cursor.execute(postgres_update_query, (training,))
 conn.commit()

 count = cursor.rowcount

 print(count, "Record updated successfully into alpaca_training")

1 Record updated successfully into alpaca_training


# delete data

In [8]:
conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

training = '牧草訓練'
postgres_delete_query = f"""DELETE FROM alpaca_training WHERE record_no = 2"""

cursor.execute(postgres_delete_query)
conn.commit()
count = cursor.rowcount

print(count, "Record delete successfully into alpaca_training")

1 Record delete successfully into alpaca_training


# delete database

In [9]:
DATABASE_URL = os.popen('heroku config:get DATABASE_URL -a dennis89linebot').read()[:-1]

conn = psycopg2.connect(DATABASE_URL, sslmode='require')
cursor = conn.cursor()

delete_table_query = '''DROP TABLE IF EXISTS alpaca_training'''

cursor.execute(delete_table_query)
conn.commit()
cursor.close()
conn.close()